In [24]:
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import sklearn

In [25]:
API_KEY = '727a54b09b424a679c79bba450f60661'

In [26]:
# Taking data from the main page and returning Sets of values for each attribute
def take_data_from_main(response):
  soup = BeautifulSoup(response.content, 'html.parser')
  title_list = soup.find_all(class_ ='list__item__content__title__name link')
  district_list = soup.select('p[class="list__item__content__subtitle"]')
  area_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--powierzchnia"] p[class = "list__item__details__icons__element__desc"]')
  number_of_rooms_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--l_pokoi"] p[class = "list__item__details__icons__element__desc"]')
  year_of_creation_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--rok_budowy"] p[class = "list__item__details__icons__element__desc"]')
  price_list = soup.select('p[class = "list__item__details__info details--info--price"]')
  href_list = soup.select('a[class="list__item__content__title__name link"]')
  return title_list, district_list, area_list, number_of_rooms_list, year_of_creation_list, price_list, href_list

# Diving into the page of an individual advertisement and returning Sets of values for each attribute which was not shown on a main page
def take_data_from_individual(link):
  soup_for_specific_advertisement = BeautifulSoup(requests.get(link).content, 'html.parser')
  type_of_building = soup_for_specific_advertisement.select('div[class="xogField xogField--rodzaj_nieruchomosci"] span[class="xogField__value"]')[0].get_text().strip()
  additional_list = soup_for_specific_advertisement.select('li[class="oglFieldList__item"] b')

  # Adding a try clause because not every offer has an information of the floor (Not given or an offer is a house)
  try:
    floor = soup_for_specific_advertisement.select('a[class="xogField xogField--pietro xogField--withIcon"] span[class="xogField__value xogField__value--big"]')[0].get_text().strip()
  except:
    floor = "nan"
  return type_of_building, additional_list, floor


def get_coordinates(address):
    API_KEY = '727a54b09b424a679c79bba450f60661'
    
    url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()
    
        # Extract the first result from the data
        result = data["features"][0]
    
        # Extract the latitude and longitude of the result
        latitude = result["geometry"]["coordinates"][1]
        longitude = result["geometry"]["coordinates"][0]
    
        return latitude, longitude
    else:
        print(f"Request failed with status code {response.status_code}")
        return None, None

In [27]:
response = requests.get('https://ogloszenia.trojmiasto.pl/nieruchomosci-rynek-pierwotny/e1i,45,f1i,1.html')
soup = BeautifulSoup(response.text, 'html.parser')
data = []
 # Using previously defined function for taking data from a main page
title_list, district_list, area_list, number_of_rooms_list, year_of_creation_list, price_list, href_list = take_data_from_main(response)
 # Iterating through lists to create separate records
for element in range(len(title_list)):
    try:
      price = price_list[element].get_text().replace(" ", "").rstrip("zł/m2")
    except:
      continue
    tytul = title_list[element].get_text().strip().replace("|", " ")
    district = district_list[element].get_text().strip()
    latitude, longitude = get_coordinates(district + " " + str(10))
    powierzchnia = area_list[element].get_text().strip().rstrip(" m2")
    number_of_rooms = number_of_rooms_list[element].get_text().strip()
    year_of_creation = year_of_creation_list[element].get_text().strip()
    price = price_list[element].get_text().replace(" ", "").rstrip("zł/m2")
    href = href_list[element]
    link = href['href']
    type_of_building, additional_list, floor = take_data_from_individual(link)

    #Converting a Set of values into a list of strings
    additional_list_str = []
    find_string = lambda x : 1 if x.lower() in additional_list_str else 0
    for item in range(len(additional_list)):
      additional_list_str.append(additional_list[item].get_text().strip().lower())

    # Using lambda function to check if an asset for a buyer is in the offer
    garage = find_string('Garaż')
    balcony = find_string('Balkon')
    internet = find_string('Internet')
    parking = find_string('Miejsce Parkingowe')
    elevator = find_string('Winda')
    terrace = find_string('Taras')
    basement = find_string('Piwnica')
    garden = find_string('Ogródek')
    #print("Garage: " + str(garage) + " Balcony: " + str(balcony) + " Internet: " + str(internet) + " Parking: " + str(parking) + " Elevator: " + str(elevator) + " Terrace: " + str(terrace) + " Basement: " + str(basement))

    # Creating a record for a specific offer and adding it to a list
    record = f"{tytul}|{district}|{latitude}|{longitude}|{powierzchnia}|{number_of_rooms}|{year_of_creation}|{price}|{type_of_building}|{garage}|{balcony}|{internet}|{parking}|{elevator}|{floor}|{terrace}|{basement}|{garden}|secondary"
    data.append(record)
    print(record)

Leszczynowy Park - mieszkanie 6.A.13 - Park na terenie osiedla!|54.3380783|18.558418226277837|41.9|2|2025|11474|Mieszkanie|1|1|1|0|1|1|0|1|0|secondary
LUNA Gdańsk Galaktyczna mieszkanie 2-pokojowe z balkonem|54.42547845|18.4859151|37.95|2|2025|10883|Mieszkanie|0|1|0|1|1|1|0|0|0|secondary
Mieszkanie 55, Budynek "I", Nowe Kowale Plus|54.3077067|18.5639208|45.58|3|2025|10150|Mieszkanie|1|1|1|1|1|4|0|1|0|secondary
Mieszkanie A4 Wieżycka Folwark 52.79m2|54.322859750000006|18.5796692932317|52.79|3|2025|10344|Mieszkanie|0|1|0|0|1|1|1|0|0|secondary
Przedsprzedaż - Gdańsk, ul. Gostyńska G2|54.347016|18.4972786|134.7|4|2026|6229|Dom bliźniak|0|0|0|0|0|nan|0|0|0|secondary
Quercus   B2.37|54.3171537|18.590914|43.17|2|2025|11229|Mieszkanie|0|0|0|0|1|3|0|0|0|secondary
Dom w zabudowie bliźniaczej stan deweloperski - Gdańsk|54.360403649999995|18.45505705712491|152.5|4|2024|8518|Dom bliźniak|0|0|0|0|0|nan|0|0|0|secondary
Nowe Południe - mieszkanie 4.A.25 - Nowość w ofercie!|54.3095959|18.6297207|36.27|

In [22]:
get_coordinates('Gdańsk Łostowice Wielkopolska 10')

Latitude: 54.3171537, Longitude: 18.590914


In [8]:
# importing geopy library and Nominatim class
from geopy.geocoders import Nominatim

# calling the Nominatim tool and create Nominatim class
loc = Nominatim(user_agent="Geopy Library")

# entering the location name
getLoc = loc.geocode("jasień potęgowska")

# printing address
print(getLoc.address)

# printing latitude and longitude
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

Potęgowska, Szadółki, Jasień, Gdańsk, województwo pomorskie, 80-175, Polska
Latitude =  54.3321107 

Longitude =  18.5521622
